In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

In [2]:
def generate_square_subsequent_mask(sz):
        r"""Generate a square mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
        """
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        mask = mask.to("cuda")
        return mask

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        x = self.dropout(x)
        x = x.to("cuda")
        return x

In [3]:
class TransformerModel(nn.Module):
    """
    
    """
    def __init__(self, d_model, nhead=2, num_decoder_layers=2, num_encoder_layers=2, ntokens=5000):
        super(TransformerModel, self).__init__()
        
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.nhead = nhead
        self.d_model = d_model
        self.ntokens = ntokens
        self.emb = nn.Embedding(self.ntokens, self.d_model)
        self.posenc = PositionalEncoding(d_model, 0.5)
        self.transformer = nn.Transformer(d_model=self.d_model, nhead=self.nhead,
                                          num_decoder_layers=self.num_decoder_layers,
                                          num_encoder_layers=self.num_encoder_layers)
        self.linear = nn.Linear(self.d_model, self.ntokens)
        
    def forward(self, x, tgt):
        x = self.emb(x) #* math.sqrt(self.d_model)
        tgt = self.emb(tgt)
        #print(x.shape, tgt.shape)
        x = self.posenc(x)
       # print(x)
        x = self.transformer(x, tgt, tgt_mask=generate_square_subsequent_mask(x.shape[0]), src_mask=generate_square_subsequent_mask(x.shape[0]))
        #return x
        return nn.functional.log_softmax(self.linear(x))
        
        
        

### This Part is from https://pytorch.org/tutorials/beginner/transformer_tutorial.html

In [5]:
import torchtext
from torchtext.data.utils import get_tokenizer

TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)

train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 16
eval_batch_size = 10
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

print(test_txt)

In [6]:
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [7]:
ntokens = len(TEXT.vocab.stoi)
ntokens

28785

In [8]:
train_data.size()

torch.Size([130419, 16])

In [9]:
model = TransformerModel(d_model=200, nhead=2, num_decoder_layers=2, num_encoder_layers=2, ntokens=ntokens)
model = model.to("cuda")

In [10]:
criterion = nn.CrossEntropyLoss()
lr = 3.0 # learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [11]:
import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        data = data.to("cuda")
        targets = targets.to("cuda")
        output = model(data, data)

        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
           # print('| epoch {:3d} | {:5d}/{:5d} batches | '
            #      'loss {:5.2f}'.format(
             #       epoch, batch, len(train_data),
              #      cur_loss))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data, data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [24]:
best_val_loss = float("inf")
epochs = 10 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

/home/main/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


| epoch   1 |   200/130419 batches | loss 15.04
| epoch   1 |   400/130419 batches | loss 31.50
| epoch   1 |   600/130419 batches | loss 678.59
| epoch   1 |   800/130419 batches | loss 1090.57
| epoch   1 |  1000/130419 batches | loss 1336.41
| epoch   1 |  1200/130419 batches | loss 3772.36
| epoch   1 |  1400/130419 batches | loss 7021.18
| epoch   1 |  1600/130419 batches | loss 9369.63
| epoch   1 |  1800/130419 batches | loss 9757.67
| epoch   1 |  2000/130419 batches | loss 14592.79
| epoch   1 |  2200/130419 batches | loss 12107.72
| epoch   1 |  2400/130419 batches | loss 19399.66
| epoch   1 |  2600/130419 batches | loss 15167.50
| epoch   1 |  2800/130419 batches | loss 17772.77
| epoch   1 |  3000/130419 batches | loss 17753.37
| epoch   1 |  3200/130419 batches | loss 15238.99
| epoch   1 |  3400/130419 batches | loss 21547.16
| epoch   1 |  3600/130419 batches | loss 26303.32
-----------------------------------------------------------------------------------------
| end 

| epoch   8 |   200/130419 batches | loss 303803.75
| epoch   8 |   400/130419 batches | loss 309105.73
| epoch   8 |   600/130419 batches | loss 315131.19
| epoch   8 |   800/130419 batches | loss 314115.20
| epoch   8 |  1000/130419 batches | loss 318878.41
| epoch   8 |  1200/130419 batches | loss 320280.94
| epoch   8 |  1400/130419 batches | loss 322594.71
| epoch   8 |  1600/130419 batches | loss 328046.06
| epoch   8 |  1800/130419 batches | loss 335126.14
| epoch   8 |  2000/130419 batches | loss 346165.71
| epoch   8 |  2200/130419 batches | loss 341073.02
| epoch   8 |  2400/130419 batches | loss 355019.08
| epoch   8 |  2600/130419 batches | loss 347089.94
| epoch   8 |  2800/130419 batches | loss 350396.21
| epoch   8 |  3000/130419 batches | loss 359521.67
| epoch   8 |  3200/130419 batches | loss 360559.98
| epoch   8 |  3400/130419 batches | loss 368141.71
| epoch   8 |  3600/130419 batches | loss 367487.87
----------------------------------------------------------------

In [25]:
test_loss = evaluate(model, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

/home/main/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


| End of training | test loss 10.29 | test ppl 29355.64
